# 🏙️ NYC Airbnb – Data Analysis Project

## Objectif
Analyser le marché Airbnb à New York afin de comprendre :
- la structure de l’offre
- les différences de prix entre boroughs
- les opportunités potentielles pour hôtes et voyageurs

Ce projet suit une logique **data analyst / data engineer junior** :
- nettoyage
- feature engineering
- modélisation analytique
- préparation pour visualisation (Looker Studio)

## 1. Environnement & librairies

Cette section initialise l’environnement de travail et importe les librairies nécessaires.


In [ ]:
import os
import kagglehub
import pandas as pd
import numpy as np
from google.colab import files

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.2f}".format)

## 2. Chargement du dataset

Le dataset utilisé est **NYC Airbnb Open Data (2019)**.


In [ ]:
path = kagglehub.dataset_download("dgomonov/new-york-city-airbnb-open-data")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'new-york-city-airbnb-open-data' dataset.
Path to dataset files: /kaggle/input/new-york-city-airbnb-open-data


In [ ]:
csv_path = os.path.join(path, "AB_NYC_2019.csv")
df = pd.read_csv(csv_path)

In [ ]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.65,-73.97,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75,-73.98,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.81,-73.94,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.69,-73.96,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.80,-73.94,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

## 3. Contrôle qualité & nettoyage

Analyse des valeurs manquantes et suppression des valeurs aberrantes afin d’assurer la fiabilité des analyses.


In [ ]:
df = df[df["price"].notna()]
df = df[df["price"] > 0]
df = df[df["price"] < 1000]  # seuil raisonnable

In [ ]:
df.isna().sum()

,0
id,0
name,16
host_id,0
host_name,21
neighbourhood_group,0
neighbourhood,0
latitude,0
longitude,0
room_type,0
price,0


In [ ]:
df["reviews_per_month"] = df["reviews_per_month"].fillna(0)

## 4. Normalisation & renommage des colonnes

Standardisation des noms de colonnes pour une meilleure lisibilité.


In [ ]:
df = df.rename(columns={
    "neighbourhood_group": "borough",
    "neighbourhood": "neighborhood"
})

## 5. Feature engineering

Création de variables analytiques pour enrichir l’analyse :
- indicateurs de prix
- disponibilité
- popularité


In [ ]:
df["price_per_night"] = df["price"]

df["is_available"] = df["availability_365"].apply(
    lambda x: 1 if x > 0 else 0
)

df["high_price_flag"] = df["price"].apply(
    lambda x: 1 if x > df["price"].median() else 0
)


In [ ]:
df["price_category"] = pd.cut(
    df["price"],
    bins=[0, 80, 200, 1000],
    labels=["Budget", "Mid-range", "Premium"]
)


In [ ]:
df["popularity_score"] = (
    df["number_of_reviews"] * df["reviews_per_month"]
)


## 6. Aperçu des données enrichies


In [ ]:
df[[
    "borough",
    "neighborhood",
    "room_type",
    "price",
    "price_category",
    "availability_365",
    "popularity_score"
]].head()


,borough,neighborhood,room_type,price,price_category,availability_365,popularity_score
0,Brooklyn,Kensington,Private room,149,Mid-range,365,1.89
1,Manhattan,Midtown,Entire home/apt,225,Premium,355,17.10
2,Manhattan,Harlem,Private room,150,Mid-range,365,0.00
3,Brooklyn,Clinton Hill,Entire home/apt,89,Mid-range,194,1252.80
4,Manhattan,East Harlem,Entire home/apt,80,Budget,0,0.90


## 7. Export du dataset nettoyé

Le dataset nettoyé est exporté pour archivage et réutilisation.


In [ ]:
df.to_csv("listings_clean.csv", index=False)
files.download("listings_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 8. Modélisation analytique

Création de tables de type **dimension / fait** pour faciliter l’analyse et la visualisation.


### 8.1 Dimension Neighborhood


In [ ]:
dim_neighborhood = (
    df[["neighborhood", "borough"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_neighborhood["neighborhood_id"] = dim_neighborhood.index + 1
dim_neighborhood.head()



,neighborhood,borough,neighborhood_id
0,Kensington,Brooklyn,1
1,Midtown,Manhattan,2
2,Harlem,Manhattan,3
3,Clinton Hill,Brooklyn,4
4,East Harlem,Manhattan,5


In [ ]:
df = df.merge(
    dim_neighborhood,
    on=["neighborhood", "borough"],
    how="left"
)


### 8.2 Table de faits – Listings


In [ ]:
fact_listings = df[[
    "id",
    "neighborhood_id",
    "price",
    "price_category",
    "room_type",
    "availability_365",
    "popularity_score"
]].rename(columns={"id": "listing_id"})

fact_listings.head()


,listing_id,neighborhood_id,price,price_category,room_type,availability_365,popularity_score
0,2539,1,149,Mid-range,Private room,365,1.89
1,2595,2,225,Premium,Entire home/apt,355,17.10
2,3647,3,150,Mid-range,Private room,365,0.00
3,3831,4,89,Mid-range,Entire home/apt,194,1252.80
4,5022,5,80,Budget,Entire home/apt,0,0.90


### 8.3 Tables agrégées

Création de tables agrégées pour analyse par :
- borough
- type de logement
- combinaison borough × room_type


In [ ]:
agg_borough = (
    df.groupby("borough")
    .agg(
        avg_price=("price", "mean"),
        listing_count=("id", "count"),
        avg_availability=("availability_365", "mean")
    )
    .reset_index()
)

agg_borough


,borough,avg_price,listing_count,avg_availability
0,Bronx,84.52,1088,165.85
1,Brooklyn,117.29,20028,99.98
2,Manhattan,177.38,21447,111.32
3,Queens,94.53,5653,144.25
4,Staten Island,96.15,370,199.32


In [ ]:
agg_room_type = (
    df.groupby("room_type")
    .agg(
        avg_price=("price", "mean"),
        listing_count=("id", "count"),
        avg_availability=("availability_365", "mean")
    )
    .reset_index()
)

agg_room_type


,room_type,avg_price,listing_count,avg_availability
0,Entire home/apt,192.95,25164,111.23
1,Private room,84.58,22268,111.06
2,Shared room,66.12,1154,162.21


In [ ]:
agg_metrics = (
    df.groupby(["borough", "room_type"])
    .agg(
        avg_price=("price", "mean"),
        listing_count=("id", "count"),
        avg_availability=("availability_365", "mean")
    )
    .reset_index()
)

agg_metrics.head()


,borough,room_type,avg_price,listing_count,avg_availability
0,Bronx,Entire home/apt,125.20,378,158.18
1,Bronx,Private room,63.15,650,171.39
2,Bronx,Shared room,59.80,60,154.22
3,Brooklyn,Entire home/apt,167.65,9510,96.84
4,Brooklyn,Private room,72.61,10107,99.77


## 9. Modification des types
Accorder les types au format français de Google Sheet.


In [ ]:
dfs = {
    "agg_metrics": agg_metrics,
    "fact_listings": fact_listings,
    "dim_neighborhood": dim_neighborhood
}

for name, df in dfs.items():
    # Identifier les colonnes float
    float_cols = df.select_dtypes(include=["float"]).columns

    # Convertir les floats en string avec virgule
    for col in float_cols:
        df[col] = (
            df[col]
            .round(2)
            .astype(str)
            .str.replace(".", ",", regex=False)
        )
    df.to_csv(
        f"{name}.csv",
        index=False,
        sep=";",
        encoding="utf-8"
    )

In [ ]:
with open("agg_metrics.csv", "r") as f:
    for _ in range(5):
        print(f.readline())


borough;room_type;avg_price;listing_count;avg_availability

Bronx;Entire home/apt;125,2;378;158,18

Bronx;Private room;63,15;650;171,39

Bronx;Shared room;59,8;60;154,22

Brooklyn;Entire home/apt;167,65;9510;96,84



## 10. Contrôles d’intégrité

Vérification de l’unicité des clés primaires.


In [ ]:
assert fact_listings["listing_id"].is_unique
assert dim_neighborhood["neighborhood_id"].is_unique


## 11. Export des données pour visualisation

Les données sont exportées dans un format compatible Google Sheets / Looker Studio.


In [ ]:
agg_metrics.to_csv(
    "agg_metrics_sheets.csv",
    index=False,
    sep=";",
    decimal=","
)
fact_listings.to_csv(
    "fact_listings_sheets.csv",
    index=False,
    sep=";",
    decimal=","
)
dim_neighborhood.to_csv(
    "dim_neighborhood_sheets.csv",
    index=False,
    sep=";"
)


In [ ]:
files.download("fact_listings.csv")
files.download("dim_neighborhood.csv")
files.download("agg_metrics.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 12. Conclusion

Ce notebook a permis de :
- nettoyer et enrichir le dataset Airbnb NYC
- structurer les données pour l’analyse
- préparer des tables exploitables pour la visualisation

Les insights sont explorés dans un dashboard Looker Studio.
